# Groq and Gradio for Realtime Voice-Powered AI Applications 🚀

In this tutorial, we'll build a voice-powered AI application using Groq for realtime speech recognition and text generation and Gradio for creating an interactive web interface.

[Groq](groq.com) is known for insanely fast inference speed that is very well-suited for realtime AI applications, providing multiple Large Language Models (LLMs) and speech-to-text models via Groq API. In this tutorial, we will use the [Distil-Whisper English](https://huggingface.co/distil-whisper/distil-large-v3) and [Llama 3.1 70B](https://github.com/meta-llama/llama-models/blob/main/models/llama3_1/MODEL_CARD.md) models for speech-to-text and text-to-text. 

[Gradio](https://www.gradio.app/) is an open-source Python library that makes it easy to prototype and deploy interactive demos without needing to write frontend code for a nice User Interface (UI), which is great if you're a developer like me who doesn't know much about frontend Bob Ross-ery. 🖌️

By combining models powered by Groq with Gradio's user-friendly interface creation, we will:

- Use Distil-Whisper English powered by Groq transcribe audio input in realtime.
- Use Llama 3.1 70B powered by Groq to generate instant responses based on the transcription.
- Create a Gradio interface to handle audio input and display results on a nice UI.

Let's get started!

## Step 1: Create a Free GroqCloud Account and Generate Your Groq API Key

If you don't already have a GroqCloud account, you can create one for free [here](https://console.groq.com) to generate a Groq API Key. We'll need the key to be able to try out the tutorial we build! 

## Step 2: Import Required Libraries

Let's import the libraries that allow us to interact with Groq API, handle audio processing, and create the Gradio interface:

In [ ]:
!pip install gradio==4.19.2
!pip install groq==0.4.1
!pip install numpy==1.26.4
!pip install soundfile==0.12.1

import gradio as gr
import groq
import io
import numpy as np
import soundfile as sf

## Step 3: Implement Audio Transcription

Let's build a function to take audio input and use Distil-Whisper English (`distil-whisper-large-v3-en`) powered by Groq to transcribe the audio:

In [ ]:
def transcribe_audio(audio, api_key):
    if audio is None:
        return ""
    
    client = groq.Client(api_key=api_key)
    
    # Convert audio to the format expected by the model
    # The model supports mp3, mp4, mpeg, mpga, m4a, wav, and webm file types 
    audio_data = audio[1]  # Get the numpy array from the tuple
    buffer = io.BytesIO()
    sf.write(buffer, audio_data, audio[0], format='wav')
    buffer.seek(0)

    bytes_audio = io.BytesIO()
    np.save(bytes_audio, audio_data)
    bytes_audio.seek(0)

    try:
        # Use Distil-Whisper English powered by Groq for transcription
        completion = client.audio.transcriptions.create(
            model="distil-whisper-large-v3-en",
            file=("audio.wav", buffer),
            response_format="text"
        )
        return completion
    except Exception as e:
        return f"Error in transcription: {str(e)}"

## Step 4: Implement Response Generation

Now, let's build a function to take the transcribed text and generate a response using Llama 3.1 70B (`llama-3.1-70b-versatile`) powered by Groq:

In [ ]:
def generate_response(transcription, api_key):
    if not transcription:
        return "No transcription available. Please try speaking again."
    
    client = groq.Client(api_key=api_key)
    
    try:
        # Use Llama 3.1 70B powered by Groq for text generation
        completion = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": transcription}
            ],
            max_tokens=150,
            temperature=0.7
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error in response generation: {str(e)}"

## Step 5: Process Audio and Response

Next, let's create a function that calls the previous two functions we built to check that a Groq API Key was provided by the user, create the transcription, and generate the response:

In [ ]:
def process_audio(audio, api_key):
    if not api_key:
        return "Please enter your Groq API key.", "API key is required."
    transcription = transcribe_audio(audio, api_key)
    response = generate_response(transcription, api_key)
    return transcription, response

## Step 6: Build Web Interface with Gradio

Finally, we'll use Gradio and the easy-to-use UI components that it provides for us to build out a simple interface for our project:

In [ ]:
# Custom CSS for the Groq badge and color scheme (feel free to edit however you wish)
custom_css = """
.gradio-container {
    background-color: #f5f5f5;
}
.gr-button-primary {
    background-color: #f55036 !important;
    border-color: #f55036 !important;
}
.gr-button-secondary {
    color: #f55036 !important;
    border-color: #f55036 !important;
}
#groq-badge {
    position: fixed;
    bottom: 20px;
    right: 20px;
    z-index: 1000;
}
"""

with gr.Blocks(theme=gr.themes.Default()) as demo:
    gr.Markdown("# 🎙️ Groq x Gradio Voice-Powered AI Assistant")
    
    api_key_input = gr.Textbox(type="password", label="Enter your Groq API Key")
    
    with gr.Row():
        audio_input = gr.Audio(label="Speak!", type="numpy")
    
    with gr.Row():
        transcription_output = gr.Textbox(label="Transcription")
        response_output = gr.Textbox(label="AI Assistant Response")
    
    submit_button = gr.Button("Process", variant="primary")
    
    # Add the Groq badge
    gr.HTML("""
    <div id="groq-badge">
        <div style="color: #f55036; font-weight: bold;">POWERED BY GROQ</div>
    </div>
    """)
    
    submit_button.click(
        process_audio,
        inputs=[audio_input, api_key_input],
        outputs=[transcription_output, response_output]
    )
    
    gr.Markdown("""
    ## How to use this app:
    1. Enter your Groq API Key in the provided field.
    2. Click on the microphone icon and speak your message (or forever hold your peace)! You can also provide a supported audio file. Supported audio files include mp3, mp4, mpeg, mpga, m4a, wav, and webm file types.
    3. Click the "Process" button to transcribe your speech and generate a response from our AI assistant.
    4. The transcription and AI assistant response will appear in the respective text boxes.
    
    """)

demo.launch()

# Conclusion
By combining Groq and Gradio, we've built a voice-powered AI assistant with just a few lines of code and learned how easy it is to create powerful, interactive AI applications! 

Feel free to experiment with this code, try different prompts, or extend the functionality to create your own personal project! 🤩